In [1]:
import numpy as np
import math
import sympy as sym
from sympy.functions.elementary.trigonometric import atan2
from scipy.integrate import ode
from sympy.utilities.lambdify import lambdify
import Kep_2_Cart
import Cart_2_Kep
import filter_functions
from IPython.core.debugger import Tracer
import warnings
import pickle
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

#MSIS: https://github.com/DeepHorizons/Python-NRLMSISE-00
#import time
from nrlmsise_00_header import *
from nrlmsise_00 import *
#SUBROUTINE GTD7D -- d[5] is the "effective total mass density
#for drag" and is the sum of the mass densities of all species
#in this model, INCLUDING anomalous oxygen.

%matplotlib inline
plt.rcParams['figure.figsize'] = (14.0, 4.0)
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
np.set_printoptions(precision=15)
sym.init_printing()
from IPython.display import display


In [2]:

#define constants
r_earth_const = 6378136.3 #meters
omega_const = 7.2921158553e-5
J_2_const = .00108262617385222
J_3_const = -.00000253241051856772
mu_earth = 3.986004415e14 #m^3/s^2


#Drag:
C_D_est = 2.0
theta_dot_const = 7.2921158553e-5 #rad/sec



In [3]:

(24*60)/90

24*60/500

In [4]:


r_earth_const = 6378136.3 #meters

circumference = 2 * math.pi * (r_earth_const + 400*1e3)

distance_per_degree = circumference/360

#print(distance_per_degree * 1e-3, ' km')

#print(distance_per_degree * 9 * 1e-3, ' km')


print(( circumference/(90*60) ) *10)

78867.19705596408


In [5]:


class SpaceObject:
    
    def __init__(self, name, ID, area, mass, a, e, i, cap_omega, w, nu):
        
        self.name = name
        self.ID = ID
        self.area = area
        self.mass = mass
        self.a = a
        self.e = e
        self.i = i
        self.cap_omega = cap_omega
        self.w = w
        self.nu = nu

        
class DateSpecs:
    
    def __init__(self, time_span, time_step, year, month, day_of_month_init, day_of_year_init,\
                              hour_init_UT):
        
        self.time_span = time_span
        self.time_step = time_step
        self.year = year
        self.month = month
        self.day_of_month_init = day_of_month_init
        self.day_of_year_init = day_of_year_init
        self.hour_init_UT = hour_init_UT

        
        
class MeasurementSpecs:
    
    def __init__(self, spaceObj, dateObj, measType):
        
        self.spaceObj = spaceObj
        self.dateObj = dateObj
        self.measType = measType



In [11]:
objectID = 1


a = r_earth_const + 400*1e3 # meters
e = 1e-7
i = math.radians(30)#40
cap_omega = math.radians(30) #90
w = 0
nu = math.radians(0)
(r_eci, v_eci) = Kep_2_Cart.COE2RV(a, e, i, cap_omega, w, nu, mu_earth)


R_pqw = eci_2_pqw(w, i, cap_omega, r_eci)
R_rsw_init = pqw_2_rsw(nu, R_pqw)



A_const = 0.9551567 #meters^2; cross-sectional area of satellite
m_const = 10 #kg; mass of satellite

spaceObj = SpaceObject('obj1', objectID, A_const, m_const, a, e, i, cap_omega, w, nu)



time_span = 60*3
time_step = 60*3

#Date of Simulation Details:
#June 24th, 2017 at 6am (this is the date & time at the beginning of the simulation/orbit)
year = 2017
month = 6
day_of_month_init = 24
day_of_year_init = 175
hour_init_local = 6
boulder_UT_offset = 6 #Boulder time + 6 hours = UT time
hour_init_UT = hour_init_local + boulder_UT_offset

dateObj = DateSpecs(time_span, time_step, year, month, day_of_month_init, day_of_year_init,\
                                 hour_init_UT)



# meas_type = 1: range and range rate
# meas_type = 2: azimuth and elevation
# meas_type = 3: azimuth, elevation, and range
meas_type = 3

measObj = MeasurementSpecs(spaceObj, dateObj, meas_type)



In [12]:
# zero density/drag


densityVal = 0
(R_eci, V_eci) = generate_measurements(measObj, densityVal)

R_eci_final = np.array(R_eci[-1])
V_eci_final = np.array(V_eci[-1])



(a, e, i, RAAN, w, nu) = Cart_2_Kep.RV2COE(R_eci_final, V_eci_final, mu_earth)
R_pqw = eci_2_pqw(w, i, RAAN, R_eci_final)
R_rsw_final = pqw_2_rsw(nu, R_pqw)

print()

alongTrack_diff_noDens = R_rsw_init[1] - R_rsw_final[1]
print(alongTrack_diff_noDens)



diff_eci = R_eci_final - r_eci
R_pqw = eci_2_pqw(w, i, RAAN, diff_eci)
print(pqw_2_rsw(nu, R_pqw))


R_eci_final_noDens = R_eci_final

Initial Orbit Altitude: 399.999322186 km
Period of Orbit: 92.560390217187 minutes
Eccentricity is ~0 and omega is undefined. Use Argument of Latitude instead.
Argument of Latitude =  0.2036460695683939

671246.285673
[    1447.190334095823346   699573.306305197416805  1187164.593111579306424]


In [14]:

#exponential density

rho_0_const = 9.518e-12 #kg/meters^3 
r_0_const = 350 * 1e3 + r_earth_const #meters 
H_const = 53.298 * 1e3 #meters

r = np.sqrt(r_eci[0]**2+r_eci[1]**2+r_eci[2]**2)
densityVal = rho_0_const*exp(-(r-r_0_const)/H_const)

(R_eci, V_eci) = generate_measurements(measObj, densityVal)

R_eci_final = np.array(R_eci[-1])
V_eci_final = np.array(V_eci[-1])


(a, e, i, RAAN, w, nu) = Cart_2_Kep.RV2COE(R_eci_final, V_eci_final, mu_earth)
R_pqw = eci_2_pqw(w, i, RAAN, R_eci_final)
R_rsw_final = pqw_2_rsw(nu, R_pqw)

print()



diff = R_eci_final - R_eci_final_noDens

R_pqw = eci_2_pqw(w, i, RAAN, diff)
print('diff btwn exp & no density:', pqw_2_rsw(nu, R_pqw)[1])

R_eci_final_expDens = R_eci_final

Initial Orbit Altitude: 399.999322186 km
Period of Orbit: 92.560390217187 minutes
Eccentricity is ~0 and omega is undefined. Use Argument of Latitude instead.
Argument of Latitude =  0.2036460695683932

diff btwn exp & no density: -5.05029822017e-09


In [15]:


#mean density over grid


densityVal = 4.76509057514e-12

(R_eci, V_eci) = generate_measurements(measObj, densityVal)

R_eci_final = np.array(R_eci[-1])
V_eci_final = np.array(V_eci[-1])


(a, e, i, RAAN, w, nu) = Cart_2_Kep.RV2COE(R_eci_final, V_eci_final, mu_earth)
R_pqw = eci_2_pqw(w, i, RAAN, R_eci_final)
R_rsw_final = pqw_2_rsw(nu, R_pqw)

print()



diff = R_eci_final - R_eci_final_expDens
R_pqw = eci_2_pqw(w, i, RAAN, diff)
print('diff btwn exp & mean density:', pqw_2_rsw(nu, R_pqw)[1])


R_eci_final_meanDens = R_eci_final


Initial Orbit Altitude: 399.999322186 km
Period of Orbit: 92.560390217187 minutes
Eccentricity is ~0 and omega is undefined. Use Argument of Latitude instead.
Argument of Latitude =  0.20364606956839282

diff btwn exp & mean density: -1.78283806347e-09


In [16]:


#max density over grid


densityVal = 8.13654280426e-12

(R_eci, V_eci) = generate_measurements(measObj, densityVal)

R_eci_final = np.array(R_eci[-1])
V_eci_final = np.array(V_eci[-1])


(a, e, i, RAAN, w, nu) = Cart_2_Kep.RV2COE(R_eci_final, V_eci_final, mu_earth)
R_pqw = eci_2_pqw(w, i, RAAN, R_eci_final)
R_rsw_final = pqw_2_rsw(nu, R_pqw)

print()

alongTrack_diff_maxDens = R_rsw_init[1] - R_rsw_final[1]
#print(alongTrack_diff_maxDens)



diff = R_eci_final - R_eci_final_meanDens
R_pqw = eci_2_pqw(w, i, RAAN, diff)
print('diff btwn mean & max density:', pqw_2_rsw(nu, R_pqw)[1])


R_eci_final_maxDens = R_eci_final

Initial Orbit Altitude: 399.999322186 km
Period of Orbit: 92.560390217187 minutes
Eccentricity is ~0 and omega is undefined. Use Argument of Latitude instead.
Argument of Latitude =  0.20364606956839224

diff btwn mean & max density: -4.01530123528e-10


In [17]:


#min density over grid


densityVal = 2.70198315652e-12

(R_eci, V_eci) = generate_measurements(measObj, densityVal)

R_eci_final = np.array(R_eci[-1])
V_eci_final = np.array(V_eci[-1])


(a, e, i, RAAN, w, nu) = Cart_2_Kep.RV2COE(R_eci_final, V_eci_final, mu_earth)
R_pqw = eci_2_pqw(w, i, RAAN, R_eci_final)
R_rsw_final = pqw_2_rsw(nu, R_pqw)

print()



diff = R_eci_final - R_eci_final_meanDens
R_pqw = eci_2_pqw(w, i, RAAN, diff)
print('diff btwn mean & min density:', pqw_2_rsw(nu, R_pqw)[1])


R_eci_final_minDens = R_eci_final

Initial Orbit Altitude: 399.999322186 km
Period of Orbit: 92.560390217187 minutes
Eccentricity is ~0 and omega is undefined. Use Argument of Latitude instead.
Argument of Latitude =  0.20364606956839323

diff btwn mean & min density: 1.52963724407e-09


In [19]:


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~max GeoMag density over grid


densityVal = 2.28623472257e-11


(R_eci, V_eci) = generate_measurements(measObj, densityVal)

R_eci_final = np.array(R_eci[-1])
V_eci_final = np.array(V_eci[-1])


(a, e, i, RAAN, w, nu) = Cart_2_Kep.RV2COE(R_eci_final, V_eci_final, mu_earth)
R_pqw = eci_2_pqw(w, i, RAAN, R_eci_final)
R_rsw_final = pqw_2_rsw(nu, R_pqw)

print()



diff = R_eci_final - R_eci_final_meanDens
R_pqw = eci_2_pqw(w, i, RAAN, diff)
print('diff btwn mean & Max GeoMag density:', pqw_2_rsw(nu, R_pqw)[1])


R_eci_final_geoMagDens = R_eci_final

Initial Orbit Altitude: 399.999322186 km
Period of Orbit: 92.560390217187 minutes
Eccentricity is ~0 and omega is undefined. Use Argument of Latitude instead.
Argument of Latitude =  0.20364606956838927

diff btwn mean & Max GeoMag density: -1.02844474456e-08


In [146]:

arcsec_per_rad = 206264.806247 #arcsec/rad
noise_rad = (1/arcsec_per_rad)  * 5 #5 arcsec noise (suggested by Moriba for a telescope)

meas3_stan_dev = .1 #meters, range

deg_in_arcsec = np.degrees(1/arcsec_per_rad)

deg_5arcsec = deg_in_arcsec * .15

station_circ = 400000 * 2 * math.pi

station_circ_metersPerDeg = station_circ/360

az_el_noise = station_circ_metersPerDeg * deg_5arcsec

print('az & el noise in meters on orbit:', az_el_noise)


print('range noise:', .1)


az & el noise in meters on orbit: 0.290888208666
range noise: 0.1


In [8]:

def x_rotation_mat(theta):
    rotation_mat = np.array([[1, 0, 0], \
                [0, math.cos(theta), -math.sin(theta)], \
                [0, math.sin(theta), math.cos(theta)]])
    return rotation_mat

def y_rotation_mat(theta):
    rotation_mat = np.array([[math.cos(theta), 0, -math.sin(theta)], \
                [0, 1, 0], \
                [math.sin(theta), 0, math.cos(theta)]])
    return rotation_mat

def z_rotation_mat(theta):
    rotation_mat = np.array([[math.cos(theta), math.sin(theta), 0], \
                [-math.sin(theta), math.cos(theta), 0], \
                [0, 0, 1]])
    #result = np.dot(rotation_mat, r)
    return rotation_mat


#ECI to PQW
def eci_2_pqw(w, i, RAAN, R_eci):
    
    step1 = np.dot(z_rotation_mat(RAAN), R_eci)
    
    step2 = np.dot(x_rotation_mat(i), step1)
    
    R_pqw = np.dot(z_rotation_mat(w), step2)

    return R_pqw


def pqw_2_rsw(nu, R_pqw):
    
    R_rsw = np.dot(z_rotation_mat(nu), R_pqw)
    
    return R_rsw
    
    
    

In [9]:


#Propogate trajectory
def orbitpropogator_noSTM(t, X_vector, density, x_sol_fcn, y_sol_fcn, z_sol_fcn):
    state = X_vector[0:3] 
    state_v = X_vector[3:6]
    
    x = state[0]
    y = state[1]
    z = state[2]

    #find X acceleration via the F(X) lambdified equation
    x_acc = x_sol_fcn(*state, *state_v, density)
    y_acc = y_sol_fcn(*state, *state_v, density)
    z_acc = z_sol_fcn(*state, *state_v, density)
    X_acc = np.array([x_acc, y_acc, z_acc])

    dx = np.array([state_v, X_acc])
    dx = dx.flatten()
    return dx




def generate_measurements(measObj, densityVal):
     


    #define constants
    r_earth_const = 6378136.3 #meters
    omega_const = 7.2921158553e-5
    J_2_const = .00108262617385222
    J_3_const = -.00000253241051856772
    mu_earth = 3.986004415e14 #m^3/s^2


    #Drag:
    A_const = measObj.spaceObj.area #meters^2; cross-sectional area of satellite
    m_const = measObj.spaceObj.mass #kg; mass of satellite
    C_D_est = 2.0
    theta_dot_const = 7.2921158553e-5 #rad/sec
    
    
    
    #Define Problem Parameters

    time_span = measObj.dateObj.time_span
    time_step = measObj.dateObj.time_step


    # meas_type = 1: range and range rate
    # meas_type = 2: azimuth and elevation
    # meas_type = 3: azimuth, elevation, and range
    meas_type = measObj.measType


    #Measurement Type 1 (Range & Range Rate) Noise:
    if meas_type == 1:
        meas1_stan_dev = .1 #meters
        meas2_stan_dev = .01 #meters/sec
    #Measurement Type 2 (Az & El) Noise: 
    if meas_type == 2:
        arcsec_per_rad = 206264.806247 #arcsec/rad
        noise_rad = (1/arcsec_per_rad)  * 5 #5 arcsec noise (suggested by Moriba for a telescope)
        meas1_stan_dev = noise_rad #radians
        meas2_stan_dev = noise_rad #radians
    if meas_type == 3:
        arcsec_per_rad = 206264.806247 #arcsec/rad
        noise_rad = (1/arcsec_per_rad)  * 5 #5 arcsec noise (suggested by Moriba for a telescope)
        meas1_stan_dev = noise_rad #radians, az
        meas2_stan_dev = noise_rad #radians, el
        meas3_stan_dev = .1 #meters, range


    #Date of Simulation Details:
    #June 24th, 2017 at 6am (this is the date & time at the beginning of the simulation/orbit)
    year = measObj.dateObj.year
    month = measObj.dateObj.month
    day_of_month_init = measObj.dateObj.day_of_month_init
    day_of_year_init = measObj.dateObj.day_of_year_init
    hour_init_UT = measObj.dateObj.hour_init_UT


    # define position and velocity vectors (meters & meters/sec)
    #satellite
    a = measObj.spaceObj.a #meters
    e = measObj.spaceObj.e
    i = measObj.spaceObj.i
    cap_omega = measObj.spaceObj.cap_omega
    w = measObj.spaceObj.w
    nu = measObj.spaceObj.nu
    (r_eci, v_eci) = Kep_2_Cart.COE2RV(a, e, i, cap_omega, w, nu, mu_earth)
    #print('Initial Position (ECI):', np.array(r_eci)/1e3, 'km')
    #print('Initial Velocity (ECI):', np.array(v_eci)/1e3, 'km')
    print('Initial Orbit Altitude:', (np.linalg.norm(r_eci)- r_earth_const)/1e3, 'km')

    period = 2*math.pi*math.sqrt(a**3/mu_earth)
    print('Period of Orbit:', period/(60), 'minutes')


    #Canbera Station (DSS 34) Australia
    lat_dss34 = math.radians(-35.398333)
    lon_dss34 = math.radians(148.981944)
    alt_dss34 = 691.75 #m

    r_ecef_dss34 = filter_functions.topo2ecef(lat_dss34, lon_dss34, alt_dss34, r_earth_const)
    #print(r_ecef_dss34)

    #Madrid Station (DSS 65) -- correct position of Madrid Station
    lat_dss65 = math.radians(40.427222)
    lon_dss65 = math.radians(355.749444)
    alt_dss65 = 834.539 #m

    r_ecef_dss65 = filter_functions.topo2ecef(lat_dss65, lon_dss65, alt_dss65, r_earth_const)
    #print(r_ecef_dss65)

    #Goldstone Station (DSS 13) California
    lat_dss13 = math.radians(35.247164)
    lon_dss13 = math.radians(243.205)
    alt_dss13 = 0 #1071.14904 #m

    r_ecef_dss13 = filter_functions.topo2ecef(lat_dss13, lon_dss13, alt_dss13, r_earth_const)
    #print(r_ecef_dss13)


    #Diego Garcia, British Indian Ocean Territory 7.41173°S 72.45222°E., Space Fence (Dedicated Sensor
    lat_diego = math.radians(-7.41173)
    lon_diego = math.radians(72.45222)
    alt_diego = 0 #m, "sea level"

    r_ecef_diego = filter_functions.topo2ecef(lat_diego, lon_diego, alt_diego, r_earth_const)


    
    
    # **Setup Dynamical Model/Equations (Position & Velocity Forecast Model)

    #Force equations: two body, J_2, & J_3
    x, y, z, J_2, J_3, r_earth, mu, r,  x_dot, y_dot, z_dot, density = \
        sym.symbols('x y z J_2 J_3 r_earth mu r,  x_dot, y_dot, z_dot, density')

    U_string = 'mu/r * ( 1 - J_2*(r_earth/r)**2 * (3/2 * (z/r)**2 - 1/2) )'
                #( 1 - J_2*(r_earth/r)**2 * (3/2 * (z/r)**2 - 1/2) ' #J_2 portion 
                #'- J_3 * (r_earth/r)**3 * ( 5/2 * (z/r)**3 - 3/2*(z/r) ) )') #J_3 portion 
    U = sym.sympify(U_string)
    U = U.subs(r, sym.sqrt(x**2+y**2+z**2))

    U_diffeq_x = U.diff(x)
    U_diffeq_y = U.diff(y)
    U_diffeq_z = U.diff(z)
    
    
    
    x_acc = '(-mu/r**3) * x'
    y_acc = '(-mu/r**3) * y'
    z_acc = '(-mu/r**3) * z'
    
    x_acc = sym.sympify(x_acc)
    y_acc = sym.sympify(y_acc)
    z_acc = sym.sympify(z_acc)
    
    
    x_acc = x_acc.subs([(r, sym.sqrt(x**2+y**2+z**2)), (mu, mu_earth)])
    y_acc = y_acc.subs([(r, sym.sqrt(x**2+y**2+z**2)), (mu, mu_earth)])
    z_acc = z_acc.subs([(r, sym.sqrt(x**2+y**2+z**2)), (mu, mu_earth)])
    

    '''
    x_acc = U_diffeq_x.subs([(r_earth, r_earth_const), (mu, mu_earth), \
                                  (J_2, J_2_const)])
    y_acc = U_diffeq_y.subs([(r_earth, r_earth_const), (mu, mu_earth), \
                                  (J_2, J_2_const)])
    z_acc = U_diffeq_z.subs([(r_earth, r_earth_const), (mu, mu_earth), \
                                  (J_2, J_2_const)])
    '''


    #Add drag to J_2 force equations
    C_D, A, m, density, theta_dot, val, val_dot = \
        sym.symbols('C_D A m density theta_dot val, val_dot')

    drag_str_x = ('-(1/2)*C_D*(A/m)*density*'   
            '(  ( 1/(sqrt((x_dot+theta_dot*y)**2 + (y_dot-theta_dot*x)**2 +z_dot**2)) ) *(x_dot+theta_dot*y))')
    
    drag_str_y = ('-(1/2)*C_D*(A/m)*density*'   
            '(  ( 1/(sqrt((x_dot+theta_dot*y)**2 + (y_dot-theta_dot*x)**2 +z_dot**2)) ) *(y_dot-theta_dot*x))')
    
    drag_str_z = ('-(1/2)*C_D*(A/m)*density*'   
            '(  ( 1/(sqrt((x_dot+theta_dot*y)**2 + (y_dot-theta_dot*x)**2 +z_dot**2)) ) *(z_dot))')
                
                
    drag_symp_x = sym.sympify(drag_str_x)
    drag_symp_y = sym.sympify(drag_str_y)
    drag_symp_z = sym.sympify(drag_str_z)

    drag_symp_x = drag_symp_x.subs([(A, A_const), (m, m_const), (C_D, C_D_est),\
                            (theta_dot, theta_dot_const)])
    drag_symp_y = drag_symp_y.subs([(A, A_const), (m, m_const), (C_D, C_D_est),\
                            (theta_dot, theta_dot_const)])
    drag_symp_z = drag_symp_z.subs([(A, A_const), (m, m_const), (C_D, C_D_est),\
                            (theta_dot, theta_dot_const)])


    drag_symp_x = drag_symp_x.subs([(r, sym.sqrt(x**2+y**2+z**2))])
    x_acc = x_acc + drag_symp_x
    
    drag_symp_y = drag_symp_y.subs([(r, sym.sqrt(x**2+y**2+z**2))])
    y_acc = y_acc + drag_symp_y
    
    drag_symp_z = drag_symp_z.subs([(r, sym.sqrt(x**2+y**2+z**2))])
    z_acc = z_acc + drag_symp_z



    #print(z_acc)

    x_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), x_acc)
    y_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), y_acc)
    z_sol_fcn = lambdify((x, y, z, x_dot, y_dot, z_dot, density), z_acc)


    
    
    
    
    
    #initializations
    t_init =  0
    t_final = time_span #seconds
    t_step = time_step
    time_array = np.arange(t_init, t_final+t_step, t_step)

    #set the initial values for the propogator:
    #reference trajectory
    y0 = np.array([r_eci, v_eci])
    y0 = y0.flatten()



    integrator = ode(orbitpropogator_noSTM)
    integrator.set_integrator('dopri5', nsteps=1e6, rtol=3e-14, atol=1e-16)
    integrator.set_initial_value(y0, t_init)
    

    result = []
    result.append(np.insert(y0, 0, t_init))
    X_vector = y0
    t = t_init
    
    

    

    while integrator.successful() and integrator.t < t_final:
        
        integrator.set_f_params(densityVal, x_sol_fcn, y_sol_fcn, z_sol_fcn)
        integrator.integrate(integrator.t + t_step)
        X_vector = integrator.y
        t = integrator.t

        result.append(np.insert(X_vector, 0, t))



    result = np.array(result)

    integrator_times = result[:, 0]
    X = result[:, [1, 2, 3]]
    V = result[:, [4, 5, 6]]
    truth_pos_vel = result[:, [1, 2, 3, 4, 5, 6]]
    pos_init = X[0]
    vel_init = V[0]

    return X, V

